In [ ]:
import os
import sys
import warnings
import pandas as pd
sys.path.append('../modules')
from transform_dst_mod import transform_dst
from transform_specimen_mod import transform_specimen

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
specimen = pd.read_csv('../Mali/data/interm/Mali_jan_1_specimen.csv')
dst = pd.read_csv('../Mali/data/interm/Mali_jan_3_dst.csv')
CaseBrowser_prod_data = pd.read_csv('../Mali/data/raw/Mali_Prod_data_01_16_24.csv')

In [ ]:
# Creating the containeridentifier so that it can be used to merge with the specimen dataframe.
containeridentifier = specimen[['patient_local_identifier', 'spec_local_identifier', 'collected_date']]

# Change the column names to match the other dataframes.
containeridentifier.rename(columns={'patient_local_identifier':'specimen', 
                                    'spec_local_identifier':'specimen local',
                                    'collected_date':'date'}, inplace=True)

In [ ]:
containeridentifier.head(20)

In [ ]:
dst.head(20)

In [ ]:
# Checking the same as above but with the data from prod
containeridentifier_prod = CaseBrowser_prod_data[['patient_local_identifier', 'private_container_identifier', 'collected_date', 'registrationdate']]

containeridentifier_prod.rename(columns={'patient_local_identifier':'specimen',
                                    'collected_date':'date'}, inplace=True)

In [ ]:
# Function that strips the dashes from the columns excpet for a certain record
def strip_dashes(df, col1, values_to_exclude):
    df[col1] = df.apply(lambda row: row[col1] if row[col1] in values_to_exclude else row[col1].replace('-', ''), axis=1)
    return df

# Values to exclude from the strip_dashes function
values_to_exclude = ['MAL092040-2', 'MAL092040-3']

# MAL09-2040-3

In [ ]:
dst = strip_dashes(dst, 'specimen', values_to_exclude)

In [ ]:
dst.head(20)

In [ ]:
# I need to change some value in the specimen column to match the values in the specimen dataframe.
dst['specimen'] = dst['specimen'].replace('MAL0920402', 'MAL092040-2')
dst['specimen'] = dst['specimen'].replace('MAL0920403', 'MAL092040-3')

In [ ]:
# Drop the specimen local column because it is not needed.
dst.drop(columns=['specimen local'], inplace=True)

In [ ]:
# I want to see how many of the specimen local identifiers are in the containeridentifier dataframe.
# I will use the isin() function to see how many are in the containeridentifier dataframe.
dst['specimen'].isin(containeridentifier['specimen']).value_counts()

In [ ]:
dst['specimen'].isin(containeridentifier_prod['specimen']).value_counts()

In [ ]:
# I want to see the records that are not in the containeridentifier dataframe.
# I will use the ~ to see the records that are not in the containeridentifier dataframe.
not_in_containerid = dst[~dst['specimen'].isin(containeridentifier['specimen'])]

In [ ]:
not_in_containerid.head(60)

In [ ]:
# not_in_containerid.to_csv('not_in_bulkupload_v2.csv', index=False)

In [ ]:
dst = pd.merge(dst, containeridentifier, on='specimen')

In [ ]:
dst.head(10)

# MAL092040-3_S_3
# MAL092040-2_S_2

In [ ]:
containeridentifier.query('specimen == "MAL0920403"')

In [ ]:
containeridentifier_prod.query('specimen == "MAL0914801"')